In [1]:
import datasets

datasets.downloads = '/home/greg/Downloads/'

trX, teX, trY, teY = datasets.mnist()

In [2]:
import numpy as np

In [22]:
decode = lambda p: np.argmax(p, axis = 1) 
labels = decode(teY)
accuracy = lambda p: np.mean(decode(p) == labels)

In [90]:
import tensorflow as tf

def dense(output_dim, x):
    shape = (x.get_shape()[1].value, output_dim)
    w = tf.Variable(tf.random_normal_initializer(0.0, 0.05)(shape))
    b = tf.Variable(tf.zeros(shape[1], dtype = tf.float32))
    return tf.matmul(x, w) + b  
    
def fit(sess, x, y, X, Y, optimizer, epochs):
    sess.run(tf.initialize_all_variables())
    N, D = X.shape

    for _ in range(epochs):
        for start in range(0, N - 32, 32):
            end = start + 32
            feed_dict = {x: X[start : end], y: Y[start : end]}
            sess.run(optimizer, feed_dict = feed_dict)

In [91]:
x = tf.placeholder(tf.float32, (None, trX.shape[1]))
y = tf.placeholder(tf.float32)
h = tf.nn.relu(dense(625, x))  
model = dense(trY.shape[1], h)
loss = tf.nn.softmax_cross_entropy_with_logits(model, y)
optimizer = tf.train.AdamOptimizer().minimize(loss)

with tf.Session() as sess:
    fit(sess, x, y, trX, trY, optimizer, epochs = 1)
    print accuracy(model.eval({x: teX}))

0.952


In [89]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(output_dim = 625, input_dim = trX.shape[1], activation = 'relu'))
model.add(Dense(output_dim = trY.shape[1], activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
model.fit(trX, trY, nb_epoch = 1)
print accuracy(model.predict_proba(teX, verbose = 0))

Epoch 1/1
60000/60000 [==============================] - 19s - loss: 0.1948    
0.9715


In [186]:
config = model.get_config()
weights = model.get_weights()

model = Sequential.from_config(config)
model.set_weights(weights)